In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import sklearn.metrics 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from langdetect import detect

In [80]:
#leemos datos 
data = pd.read_csv('itamplify_train.csv')
data

,X,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,1000,4pR4oQJULf7FDB54TleMyH,Limoblaze;Lecrae;Happi,Jireh (My Provider),Jireh (My Provider),64,168000,0,0.443,0.778,...,-7.564,0,0.2660,0.241000,0.000000,0.2150,0.6280,128.250,5,afrobeat
1,1001,4Ds4bq6aqOSAp1T7DikOi4,Criolo,Ainda Há Tempo,Ainda Há Tempo,44,318360,0,0.580,0.800,...,-7.528,1,0.3450,0.281000,0.000000,0.0708,0.4830,81.303,4,afrobeat
2,1002,7nYbKsvhM88mHCmFsAIQVp,Criolo,Fellini,Fellini,42,171989,0,0.805,0.746,...,-5.211,0,0.1850,0.238000,0.000000,0.2150,0.7170,127.922,4,afrobeat
3,1003,7gVNP7rI9UBZndge0ulKfL,Criolo,Nó na Orelha,Lion Man,46,205440,0,0.582,0.660,...,-4.988,0,0.0544,0.399000,0.000000,0.1270,0.4840,83.789,4,afrobeat
4,1004,1hLvWelTny8vttEEZIXVjw,Criolo,Espiral de Ilusão,Espiral de Ilusão,44,220800,0,0.776,0.314,...,-9.513,1,0.0340,0.881000,0.000723,0.1390,0.4760,124.055,4,afrobeat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8261,109993,1t0nwk5GRdT3frkWp15qSl,Akina Nakamori,コンプリート・シングル・コレクションズ〜ファースト・テン・イヤーズ<ライノ・プレミアム・エデ...,1/2の神話,25,201133,0,0.513,0.860,...,-5.969,0,0.0907,0.010900,0.027900,0.1090,0.4560,164.913,4,techno
8262,109994,1zsrSsPEhdLSUDujjmFhgD,Endymion,Wide Awake,Wide Awake,25,204387,0,0.396,0.937,...,-2.527,0,0.0677,0.077300,0.000000,0.0889,0.1410,155.041,4,techno
8263,109995,1910z5NqrGoJ8SuKWNu19D,Scooter,I Want You to Stream!,Bora Bora Bora - Live,25,230250,0,0.726,0.875,...,-5.957,0,0.1930,0.203000,0.001510,0.5680,0.5230,139.955,4,techno
8264,109997,3Rn21K5Te3TUM0AcFtisuX,Marshall Masters,Thunderdome Die Hard II (Digital Version),Return To Zero - The Mystery Land Anthem,25,120110,0,0.679,0.866,...,-9.691,1,0.0719,0.000038,0.755000,0.9560,0.0457,147.086,4,techno


In [81]:
#Si la canción es una colaboración (n de artistas)
semicolon_count = []

for i in range(len(data)):
    try:
        text = data['artists'][i]
        count = text.count(";")
        semicolon_count.append(count)
    except:
        semicolon_count.append(0)

data['collab'] = semicolon_count

In [83]:
data['len_title'] = data['track_name'].apply(len)
data['n_words_title'] = data['track_name'].apply(lambda x: x.count(' '))+1

In [84]:
#Detectamos el idioma y creamos una nueva columna 
language = []
for i in range(len(data)):
    try:
        lan = detect(data['track_name'][i])
        language.append(lan)
    except:
        # Handle the case where language detection fails
        language.append("Unknown")

data['language'] = language

In [85]:
# Define the language array
languages = ['en', 'pt', 'it', 'hr', 'vi', 'id', 'hu', 'ro', 'es', 'et', 'nl',
             'sw', 'no', 'fr', 'tl', 'cs', 'so', 'tr', 'ca', 'da', 'sk', 'de',
             'fi', 'sl', 'pl', 'sv', 'cy', 'lt', 'af', 'Unknown', 'lv', 'ja',
             'sq', 'ru', 'ko', 'uk', 'mk', 'zh-cn', 'zh-tw']

# Get dummy variables
language_dummies = pd.get_dummies(data['language'])

# Ensure all languages in the array are represented in the dummy variables
for language in languages:
    if language not in language_dummies.columns:
        language_dummies[language] = 0

# Reorder columns to match original languages array
language_dummies = language_dummies[languages]

# Assign dummy variables back to DataFrame
data = pd.concat([data, language_dummies], axis=1)

In [87]:
data2= data.drop(['X', 'track_id', 'artists', 'album_name', 'track_name','language'], axis=1)

In [88]:
train, test = train_test_split(data2, 
                                stratify=data['track_genre'],
                                test_size=0.2,
                                random_state=42)

train, val = train_test_split(train,
                                stratify=train['track_genre'],
                                test_size=0.2,
                                random_state=42)


X_train = train.drop(columns=['track_genre'])
y_train = train['track_genre']
X_test = test.drop(columns=['track_genre'])
y_test = test['track_genre']



In [89]:
#RANDOM FOREST 
rfc = RandomForestClassifier(n_estimators=100, random_state=42)


rfc.fit(X_train, y_train)


predictions = rfc.predict(X_test)


accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
f1 = f1_score(y_test, predictions, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

print("\nClassification Report:")
print(classification_report(y_test, predictions))

Accuracy: 0.6293833131801693
Precision: 0.6224464325612035
Recall: 0.6293833131801693
F1 Score: 0.6154788685669956

Classification Report:
              precision    recall  f1-score   support

    afrobeat       0.55      0.31      0.40        68
 alternative       0.51      0.38      0.43        72
       blues       0.36      0.23      0.28       120
   classical       0.80      0.70      0.74        50
     country       0.60      0.31      0.41        58
       disco       0.58      0.60      0.59       139
        folk       0.49      0.61      0.55       153
        funk       0.76      0.68      0.72       126
 heavy-metal       0.68      0.87      0.76        84
     hip-hop       0.59      0.85      0.70       115
        jazz       0.52      0.26      0.35        54
  psych-rock       0.55      0.66      0.60       172
   reggaeton       0.68      0.54      0.60        74
       samba       0.80      0.89      0.84       195
      techno       0.77      0.79      0.78       